# Statistical modeling

In this notebook we will see how user defined functions can be used for statistical modeling using [scipy](http://scipy.github.io/devdocs/reference/index.html) package. We will also see how to implement Pandas UDF which has better performace than vanilla UDF because it can laverage [Apache Arrow](https://arrow.apache.org/) under the hood for exchanging the data and vectorized execution that is supported by the scipy package.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lit, count, year, pandas_udf, avg

from pyspark.sql.types import IntegerType

import os

from scipy.stats import poisson
import pandas as pd

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('UDFs I')
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

answers_input_path = os.path.join(project_path, 'data/answers')

## Task
For each user compute probability that the user is going to answer 5 questions in the next year. Use simple model based on poisson distribution.

* Create a DataFrame with two cols: user_id, answers, where the second is the average number of questions the user answered per year.
* Implement UDF that will use poisson distribution from scipy package to compute the probability that if the user answered n questions per year, he will answer 5 questions in the next year
* Implement the UDF again, but this time as Pandas UDF

In [ ]:
# we will need answers dataset:

answersDF = (
    spark
    .read
    .option('path', answers_input_path)
    .load()
)

## Create input DataFrame
* filter for rows where user_id is not null
* compute average number of answers per user per year
* group by user and year
* use count to see how many questions each user answered in each year
* group by again but now only per user
* compute the average per year for each user

In [ ]:
input_df = (
    answersDF
    .filter(col('user_id').isNotNull())
    .withColumn('creation_year', year('creation_date'))
    .groupBy(
        'creation_year', 'user_id',
    )
    .agg(
        count('*').alias('answers')
    )
    .groupBy('user_id')
    .agg(
        avg('answers').alias('answers')
    )
)

In [ ]:
input_df.show(n=5)

## Define a python function
### Hint:

* it should take as argument year average and return the probability that `k` questions will be answered in next year
* use [pmf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html#scipy.stats.poisson) function of the poisson in scipy
* define `k` to be a constant equal 5
* test if the function works

In [ ]:
# your code here

k = 5

def get_probability(year_average):
    return poisson.pmf(k, year_average)

In [ ]:
get_probability(6)

## Define the UDF:
### Hint:

* once you have the python function, make the UDF from it. See udf in [docs](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.udf.html#pyspark.sql.functions.udf)
* the return type will be float, since we will compute probability
* make sure to use the `float()` function for the return value to cast it to float

In [ ]:
@udf('float')
def get_probability(year_average):
    return float(poisson.pmf(k, year_average))

## Apply the udf

In [ ]:
(
    input_df
    .withColumn('probability', get_probability(col('answers')))
).show(n=5)

## Try it with Pandas
* create local Pandas dataframe with input data
* pass a local Pandas series to poisson to see what it returns
* define a function that will take pandas series as input argument and will return also pandas series

### Hint:

* create a pandas series from pandas dataframe as `local_data['answers']`, where local_data is pd_df


In [ ]:
local_data = input_df.toPandas()

In [ ]:
# It returns numpy array

poisson.pmf(k, local_data['answers'])

In [ ]:
# We can easily create a pandas series from it:

pd.Series(poisson.pmf(k, local_data['answers']))

In [ ]:
def get_probability_pd(year_average):
    return pd.Series(poisson.pmf(k, year_average))

In [ ]:
get_probability_pd(local_data['answers'])

### Hint

* Once you have the function make a pandas udf from it
* See [pandas_udf](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.pandas_udf.html#pyspark.sql.functions.pandas_udf) in the docs

In [ ]:
# define pandas udf 

@pandas_udf('float')
def get_probability_pd(year_average):
    return pd.Series(poisson.pmf(k, year_average))

In [ ]:
# Apply the UDF:

(
    input_df
    .withColumn('probability', get_probability_pd(col('answers')))
).show(n=5)

## Compare the performace for both UDFs
### Hint

* run the query with the noop format
* check the execution time in SparkUI

In [ ]:
# execution of vanilla UDF:

(
    input_df
    .withColumn('probability', get_probability(col('answers')))
    .write
    .mode('overwrite')
    .format('noop')
    .save()
)

In [ ]:
# execution of Pandas UDF:

(
    input_df
    .withColumn('probability', get_probability_pd(col('answers')))
    .write
    .mode('overwrite')
    .format('noop')
    .save()
)